# Mesh generation function

In [1]:
import numpy as np

def generate_mesh_square(node_count, length, mesh_type='quad', fixed_node_option='all'):
    """
    Generate a square mesh with specified properties.

    Parameters:
        node_count (int): Number of nodes per side (>= 2).
        length (float): Total length of the square side (> 0).
        mesh_type (str): 'quad' for quadrilateral, 'tri' for triangular, or 'tri_both' for both diagonals in each quad.
        fixed_node_option (str): Controls fixed nodes:
            'all' - All edge nodes fixed.
            'everyOther' - Every other edge node fixed.
            'none' - No nodes fixed.

    Returns:
        verts (numpy.ndarray): Nodal coordinates (n x 3 array).
        edges (numpy.ndarray): Connectivity matrix (m x 2 array).
        fixed (numpy.ndarray): Fixed nodes (p x 3 array).
        fixed_indices (numpy.ndarray): Indices of fixed nodes.
        midlines (dict): Indices for horizontal and vertical midlines.
        diagonals (dict): Indices for main and anti-diagonals.
    """
    # Input validation
    if node_count < 2:
        raise ValueError("node_count must be at least 2.")
    if length <= 0:
        raise ValueError("length must be greater than 0.")
    if mesh_type not in {'quad', 'tri', 'tri_both'}:
        raise ValueError('mesh_type must be "quad", "tri", or "tri_both".')
    if fixed_node_option not in {'all', 'everyOther', 'none'}:
        raise ValueError('fixed_node_option must be "all", "everyOther", or "none".')

    # Step 1: Create nodes
    spacing = length / (node_count - 1)
    x, y = np.meshgrid(np.linspace(0, length, node_count), np.linspace(0, length, node_count))
    verts = np.column_stack((x.ravel(), y.ravel(), np.zeros(node_count**2)))  # All nodes in the z=0 plane initially

    # Step 2: Define connectivity
    edges = []
    for i in range(node_count - 1):
        for j in range(node_count - 1):
            # Get indices of the current square's corners
            n1 = i * node_count + j
            n2 = n1 + 1
            n3 = n1 + node_count
            n4 = n3 + 1

            if mesh_type == 'quad':
                # Quadrilateral connectivity
                edges.extend([(n1, n2), (n2, n4), (n4, n3), (n3, n1)])
            elif mesh_type == 'tri':
                # Triangular connectivity
                edges.extend([(n1, n2), (n2, n4), (n4, n1), (n1, n3), (n3, n4)])
            elif mesh_type == 'tri_both':
                # Triangular connectivity (both diagonals per quad)
                edges.extend([(n1, n2), (n2, n4), (n4, n1)])  # Triangle 1
                edges.extend([(n2, n3), (n3, n4), (n4, n2)])  # Triangle 2

    # Remove duplicate edges
    edges = np.unique(np.sort(edges, axis=1), axis=0)

    # Step 3: Determine fixed nodes
    edge_indices = np.unique(
        np.concatenate([
            np.arange(node_count),  # Bottom edge
            np.arange(node_count - 1, node_count**2, node_count),  # Right edge
            np.arange(node_count**2 - 1, node_count**2 - node_count - 1, -1),  # Top edge
            np.arange(node_count**2 - node_count, -1, -node_count)  # Left edge
        ])
    )

    if fixed_node_option == 'all':
        fixed_indices = edge_indices
    elif fixed_node_option == 'everyOther':
        fixed_indices = edge_indices[::2]
    else:  # 'none'
        fixed_indices = np.array([], dtype=int)
    
    fixed = verts[fixed_indices]

    # Step 4: Identify midlines and diagonals
    midlines = {
        'horizontal': np.where(np.abs(verts[:, 1] - length / 2) < spacing / 2)[0],
        'vertical': np.where(np.abs(verts[:, 0] - length / 2) < spacing / 2)[0]
    }

    diagonals = {
        'main': np.where(np.abs(verts[:, 0] - verts[:, 1]) < spacing / 2)[0],
        'anti': np.where(np.abs(verts[:, 0] + verts[:, 1] - length) < spacing / 2)[0]
    }

    return verts, edges, fixed, fixed_indices, midlines, diagonals


# Mesh deformation tool - take the nodes and move them

In [2]:
import numpy as np
import matplotlib.pyplot as plt

def apply_shape_function(verts, lines, shape_function, max_height, plot_option=False):
    """
    Applies a shape function to the vertices along specified lines.

    Parameters:
        verts (numpy.ndarray): Vertex coordinates (n x 3 array).
        lines (dict): Dictionary with diagonal indices or other line definitions.
        shape_function (str): Shape function ('pyramid', 'gaussian', 'sinusoidal').
        max_height (float): Maximum height for the shape function.
        plot_option (bool): If True, plot before and after the transformation.

    Returns:
        updated_verts (numpy.ndarray): Modified vertex coordinates (n x 3 array).
        new_fixed_indices (numpy.ndarray): Indices of updated vertices.
    """
    # Validate shape function
    valid_shapes = {'pyramid', 'gaussian', 'sinusoidal'}
    if shape_function not in valid_shapes:
        raise ValueError(f"Invalid shape function. Choose from {valid_shapes}.")

    # Combine all line indices into a single list
    line_indices = np.unique(np.concatenate([lines[key] for key in lines]))

    new_fixed_indices = line_indices

    # Extract vertices corresponding to the line indices
    line_verts = verts[line_indices]

    # Calculate distances from the line's center for each vertex
    line_center = np.mean(line_verts[:, :2], axis=0)  # Center in the x-y plane
    distances = np.linalg.norm(line_verts[:, :2] - line_center, axis=1)

    # Normalize distances to [0, 1]
    max_distance = np.max(distances)
    normalized_distances = distances / max_distance

    # Compute new z values based on the selected shape function
    if shape_function == 'pyramid':
        # Linear height distribution (inverted V shape)
        new_z = max_height * (1 - normalized_distances)
    elif shape_function == 'gaussian':
        # Gaussian curve
        new_z = max_height * np.exp(-4 * (normalized_distances ** 2))
    elif shape_function == 'sinusoidal':
        # Half sinusoidal wave (arch-like shape)
        new_z = max_height * (np.sin(np.pi * (1 - normalized_distances) / 2) ** 2)
    else:
        raise ValueError("Unsupported shape function.")

    # Assign new z values to the vertices
    updated_verts = verts.copy()
    updated_verts[line_indices, 2] = new_z

    # Plot before and after if plot_option is enabled
    if plot_option:
        fig = plt.figure(figsize=(12, 6))

        # Original vertices
        ax1 = fig.add_subplot(121, projection='3d')
        ax1.scatter(verts[:, 0], verts[:, 1], verts[:, 2], c='b', s=20, label="Original")
        ax1.scatter(line_verts[:, 0], line_verts[:, 1], line_verts[:, 2], c='r', s=50, label="Line")
        ax1.set_title("Original Mesh")
        ax1.set_xlabel("X")
        ax1.set_ylabel("Y")
        ax1.set_zlabel("Z")
        ax1.grid(True)
        ax1.legend()
        ax1.view_init(elev=20, azim=30)

        # Updated vertices
        ax2 = fig.add_subplot(122, projection='3d')
        ax2.scatter(updated_verts[:, 0], updated_verts[:, 1], updated_verts[:, 2], c='b', s=20, label="Updated")
        ax2.scatter(updated_verts[line_indices, 0], updated_verts[line_indices, 1], updated_verts[line_indices, 2],
                    c='r', s=50, label="Updated Line")
        ax2.set_title(f"Mesh After Applying {shape_function.capitalize()} Shape")
        ax2.set_xlabel("X")
        ax2.set_ylabel("Y")
        ax2.set_zlabel("Z")
        ax2.grid(True)
        ax2.legend()
        ax2.view_init(elev=20, azim=30)

        plt.tight_layout()
        plt.show()

    return updated_verts, new_fixed_indices


# generate a mesh using the above:

In [ ]:
node_count= 10
node_length= 100
mesh_type = 'quad'
fixed_node_option = 'all'

verts, edges, fixed, fixed_indices, midlines, diagonals = generate_mesh_square(node_count,node_length,mesh_type,fixed_node_option)

#get rest lengths here
edge_vectors = verts[edges[:, 1]] - verts[edges[:, 0]]
rest_lengths = np.linalg.norm(edge_vectors, axis=1, keepdims=True)  # rest lengths of edges
#print(rest_lengths)

shape_function = 'gaussian'
max_height = 50
verts, new_fixed_indices = apply_shape_function(verts, diagonals, shape_function, max_height, plot_option=True)
#fixed = verts[new_fixed_indices,:]


AllFixedIndices = np.unique(np.hstack((fixed_indices,new_fixed_indices)))
AllIndices = (np.arange(len(verts)))
AllFreeIndices = np.array(list((set(AllIndices) | set(AllFixedIndices)) - (set(AllIndices) & set(AllFixedIndices))))
#print(AllFreeIndices)
vertexCount = len(verts)
#print(vertexCount)
AppliedLinkForce = np.zeros((vertexCount, 3))
AllFixed = verts[AllFixedIndices]

# DR params
params = {
    'MaximumIterations': 150,
    'tolerance': 5e-1,
    'dt': 0.49,  # Time interval
    'm': 1,  # Fictitious mass
    'Elastic': 1000,  # Elastic modulus (MPa)
    'Area': (0.1**2) * np.pi,  # Cross-sectional area (note: use ** for exponentiation)
    'ConstantForce': [0, 0, 0],  # External forces
    'meshType': 'triangular_prismatic',  # Mesh type
    'length_threshold': 0.5,  # Threshold for long edges
    'restlengthOG': rest_lengths,
}

LinkTension, xyz, KEAllStore, edges, history = dynamic_relaxation(params,verts, edges, AllFreeIndices, AllFixed, AppliedLinkForce)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_geometry(xyz, edges, fixed):
    """
    Plot geometry in 3D.

    Parameters:
    - xyz: np.array of shape (N, 3), node coordinates.
    - edges: np.array of shape (M, 2), indices of edges connecting nodes.
    - fixed: np.array of shape (P, 3), coordinates of fixed nodes.
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot edges
    for edge in edges:
        x_vals = [xyz[edge[0], 0], xyz[edge[1], 0]]
        y_vals = [xyz[edge[0], 1], xyz[edge[1], 1]]
        z_vals = [xyz[edge[0], 2], xyz[edge[1], 2]]
        ax.plot(x_vals, y_vals, z_vals, 'b')  # Blue lines for edges
    
    
    # Plot free nodes
    #ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2], c='k', marker='o', label='Nodes')
    
    # Plot fixed nodes
    ax.scatter(fixed[:, 0], fixed[:, 1], fixed[:, 2], c='r', marker='*', label='Fixed Nodes')


    
    # Labels, title, and grid
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Geometry')
    ax.legend()
    ax.grid(True)
    ax.set_box_aspect([1, 1, 1])  # Equal scaling

    plt.show()

plot_geometry(xyz, edges, AllFixed)

In [ ]:
history

# AI written finished steps from below:

In [24]:
import numpy as np

def dynamic_relaxation(params, xyz, edges, free_indices, fixed, applied_link_force):
    """
    Perform dynamic relaxation to find the equilibrium state of a structure.
    Returns:
        LinkTension, xyz, KEAllStore, edges, history
    """

    # Extract parameters
    dt = params['dt']
    area = params['Area']
    E_elastic = params['Elastic']
    restlengthOG = params['restlengthOG']
    tolerance = params['tolerance']
    max_iterations = params['MaximumIterations']
    applied_constant_force = params['ConstantForce']

    # Initialize variables
    vertex_count = xyz.shape[0]
    edge_count = edges.shape[0]
    K_S = (E_elastic * area) / restlengthOG  # Initial geometric stiffness

    v = np.zeros((vertex_count, 3))  # Initializing velocities
    S = np.zeros((vertex_count, 3))  # Forces on nodes
    R = np.zeros((vertex_count, 3))  # Residual forces
    KE = 0  # Initial kinetic energy
    reset_flag = False

    # Initialize history
    history = {
        "xyz": [],
        "v": [],
        "KE": [],
        "iters": 0,
    }

    # Main loop variables
    t = 1
    difference = np.inf
    KE_store = []

    while difference > tolerance and t <= max_iterations:
        # Step 1: Initialization
        vp = np.copy(v)  # Store previous velocities
        xyz0 = np.copy(xyz)  # Store previous positions
        KE_0 = KE  # Store previous kinetic energy

        # Step 2: Compute Link Lengths and Forces
        link_vectors_t = xyz[edges[:, 1]] - xyz[edges[:, 0]]
        link_lengths_t = np.linalg.norm(link_vectors_t, axis=1, keepdims=True)
        print(link_lengths_t)

        link_tension = np.full_like(link_lengths_t, applied_constant_force[2])  # Initialize constant link tension
        link_tension += K_S * (link_lengths_t - restlengthOG)
        link_tension[link_tension < 0] = 0  # No compression allowed

        # Step 3: Node-Specific Mass Computation
        m = np.zeros(vertex_count)
        geom_stiffness = link_tension / link_lengths_t
        for node in range(vertex_count):
            adjoining_edges = np.sum(edges == node, axis=1)
            stiffness = np.sum((K_S / restlengthOG) * adjoining_edges + geom_stiffness * adjoining_edges)
            m[node] = stiffness

        # Step 4/5: Force Resolution
        edge_force_s = np.zeros((edge_count, 3))
        for i in range(edge_count):
            edge_force_s[i] = (link_vectors_t[i] / link_lengths_t[i]) * link_tension[i]
            S[edges[i, 0]] += edge_force_s[i]
            S[edges[i, 1]] -= edge_force_s[i]

        R[free_indices] = applied_link_force[free_indices] + S[free_indices]

        # Step 6: Update Velocities and Positions
        A = 1  # Damping constant
        B = (dt / m[:, None])

        v[free_indices] = A * vp[free_indices] + B[free_indices] * R[free_indices]
        xyz[free_indices] = xyz0[free_indices] + dt * v[free_indices]

        KE = np.sum(0.5 * m * np.linalg.norm(v, axis=1)**2)
        KE_store.append(KE)

        # Step 7/8: Kinetic Energy Correction
        if t >= 2 and not reset_flag and KE > KE_0:
            E = KE_store[-2] - KE_store[-1]
            D = KE_store[-3] - KE_store[-2]
            q = max(0, min(E / (E - D), 1))  # Clamp q between 0 and 1

            R_t = m[:, None] * (v - vp) / dt
            xyz -= (dt * (1 + q) * vp) + (dt**2 / 2) * q * (R_t / m[:, None])

            # Reset variables
            KE = 0
            v = np.zeros((vertex_count, 3))
            R = np.zeros((vertex_count, 3))
            S = np.zeros((vertex_count, 3))
            reset_flag = True
        else:
            reset_flag = False

        # Step 9: Convergence Check
        difference = np.linalg.norm(xyz - xyz0, ord='fro')
        history["xyz"].append(np.copy(xyz))
        history["v"].append(np.copy(v))
        history["KE"].append(KE)

        t += 1

    history["iters"] = t - 1
    return link_tension, xyz, KE_store, edges, history


# Attempt to write the matlab code into python explicitely 

In [ ]:
def dynamic_relaxation(params, xyz, edges, freeIndices, fixed, AppliedLinkForce):
    ''' 
    return LinkTension, xyz, KEAllStore, edges, history
    '''

    # Extract parameters
    dt = params['dt']
    Area = params['Area']
    E_elastic = params['Elastic']
    restlengthOG = params['restlengthOG']
    tolerance = params['tolerance']
    MaximumIterations = params['MaximumIterations']
    AppliedConstantForce = params['ConstantForce']  # i need to figure out if this should e dimensional ie.e 3rd dim = z = grav = -9.81

    # Initialize variables
    vertex_count = len(verts)
    edge_count = len(edges)
    K_S =  (E_elastic * Area) / rest_lengths #initial derivation of geometric stiffness

    v = np.zeros(vertex_count, 3)  #Initializing velocities
    S = np.zeros(vertex_count, 3) # Forces on nodes
    R = np.zeros(vertex_count, 3) # Residual forces
    KE = 0 # Initial kinetic energy
    ResetFlag = 0

    #KEAllStore = {}; # Store kinetic energy for all iterations
    # Initialize history dictionary
    history = {
        "xyz": [],  # List to store vertex positions at each iteration
        "v": [],    # List to store velocities at each iteration (optional)
        "KE": [],   # List to store kinetic energy at each iteration
        "iters": 0  # Total number of iterations
        }

    ## Main loop
    t = 1
    difference = np.inf
    #KEStore = []

    while (difference > tolerance) and (t <= MaximumIterations):
        ## Step 1: Initialization
        vp = v # Store previous velocities
        xyz0 = xyz;  # Store previous positions
        KE_0 = KE;  # Store previous kinetic energy

        ### Step 2: Compute Link Lengths and Forces 
        LinkVectors_t = xyz[edges[:, 1]] - xyz[edges[:, 0]]
        LinkLengths_t = np.linalg.norm(LinkVectors_t, axis=1, keepdims=True)

        LinkTension = np.repeat(AppliedConstantForce,[length(LinkLengths_t), 1])   #initialise constant link tensions? 
        LinkTension = LinkTension + K_S * (LinkLengths_t - restlengthOG)  # Compute tensions
        LinkTension[LinkTension < 0] = 0 #disallow compression

        ## Step 3: Node-Specific Mass Computation
        m = np.zeros(vertex_count, 1)
        GeomStiffness = (LinkTension / LinkLengths_t)
        for nodes in np.arange(1,vertex_count+1,1):
            AdjoiningEdges = sum(edges == nodes, 2)
            stiffness = sum((K_S / restlengthOG) * AdjoiningEdges + GeomStiffness * AdjoiningEdges)
            m[nodes] = stiffness

        ## Step 4/5: Force Resolution
        #edge_force_vectors = spring_forces * (edge_vectors_t / edge_lengths_t)
        EdgeForceS = np.zeros(edge_count, 3)
        for i in np.arange(1,edge_count+1,1):
            for j in np.arange(1,3,1):
                EdgeForceS(i,j) = LinkVectors_t(i,j) / LinkLengths_t(i) * LinkTension(i)

            S[edges[i,1],:] = S[edges[i,1],:] + EdgeForceS[i,:]
            S[edges(i,2),:] = S[edges[i,2],:] - EdgeForceS[i,:]

        R[freeIndices,:] = AppliedLinkForce[freeIndices,:] + S[freeIndices,:]

        ## Step 6: Update Velocities and Positions ( and some parameters) if using damping methodloogy
        A = 1 # Damping constant
        B = np.repeat((dt / m), [1, 3])

        v[freeIndices,:] = A * vp[freeIndices,:] + B[freeIndices] * R[freeIndices,:]
        xyz(freeIndices,:) = xyz0(freeIndices,:) + dt * v(freeIndices,:)

        KE = sum(0.5 * m .* vecnorm(v, 2, 2).^2)
        KEStore(t) = KE

        ## Step 7/8: Kinetic Energy Correction
        if t <= 2 || ResetFlag == 1
            ResetFlag = 0;
        else
            if KE > KE_0 && ResetFlag == 0
                # Backtracking correction
                E = KEStore(t-1) - KEStore(t);
                D = KEStore(t-2) - KEStore(t-1);
                q = max(0, min(E / (E - D), 1)); # Clamp q between 0 and 1
                
                R_t = (m .* (v - vStore{t-1})) / dt;
                xyz = xyz - (dt * (1 + q) * vStore{t-1}) + (dt^2 / 2) * q * (R_t ./ m);

                # Reset variables
                KE = 0;
                v = zeros(vertexCount, 3);
                R = zeros(vertexCount, 3);
                S = zeros(vertexCount, 3);
                ResetFlag = 1;
                warning(['Timestep ', num2str(t), ': KE spike, resetting process']);
            end
        end

        ## Step 9: Convergence Check
        difference = norm(xyz - xyz0, 'fro');
        vStore{t} = v; # Store velocities for backtracking
        history.xyz{t} = xyz; # Store position history
        history.v{t} = v; # Store velocity history
        history.KE{t} = KE; # Store kinetic energy history
        t = t + 1;
    end
    #store how many iterations:
    history.iters = t-1;
    # Store final kinetic energy history
    KEAllStore = KEStore;

end

# Dynamic relaxation with KE, NOT WORKING

In [ ]:
# Input Data
#verts = np.array(v)  # list of initial vertex positions as xyz coordinates
#edges = np.array(e)  # list of edges as pairs of vertex indices
#fixed = list(fixed_indices)  # list of fixed vertex indices
externalLoad = np.zeros((len(verts), 3))  # external load on each vertex (e.g., gravity)

# Parameters
E_elastic = 1000  # Hooke's law elasticity constant
dt = 0.59  # time step for the simulation
m = 2  # mass of each vertex
tolerance = 1e-5  # kinetic energy convergence tolerance
max_iterations = 200  # maximum number of iterations
Area = (0.1**2) * np.pi # assumed Cross-sectional area

# Derived Data
free_indices = AllFreeIndices#verts[~fixed_indices,:]#list(set(range(len(verts))) - set(fixed))  # indices of free vertices
#print(free_indices)
vertex_count = len(verts)
edge_count = len(edges)
fixed_indices = AllFixedIndices 
K = (E_elastic * Area) / rest_lengths #initial derivation of geometric stiffness

# Initialize state variables
xyz = np.copy(verts)  # current vertex positions
v = np.zeros((vertex_count, 3))  # velocity of vertices
#edge_vectors = xyz[edges[:, 1]] - xyz[edges[:, 0]]
#rest_lengths = np.linalg.norm(edge_vectors, axis=1, keepdims=True)  # rest lengths of edges
p = np.copy(externalLoad)  # external forces on vertices

# Initialize history dictionary
history = {
    "xyz": [],  # List to store vertex positions at each iteration
    "v": [],    # List to store velocities at each iteration (optional)
    "KE": [],   # List to store kinetic energy at each iteration
    "iters": 0  # Total number of iterations
}


# Initialize tracking variables
vStore = []
KEStore = []
ResetFlag = False
xyz_prev = xyz.copy()

# Main DR Loop
iteration = 0
kinetic_energy_prev = np.inf

while iteration < max_iterations:
    # Compute forces on edges
    edge_vectors_t = xyz[edges[:, 1]] - xyz[edges[:, 0]]
    edge_lengths_t = np.linalg.norm(edge_vectors_t, axis=1, keepdims=True)
    spring_forces = k * (edge_lengths_t - rest_lengths) #todo - apply linktension i.e add fixed num? set less 0 = 0
    spring_forces[spring_forces < 0] = 0 
    edge_force_vectors = spring_forces * (edge_vectors_t / edge_lengths_t)

    # # Adaptive mass computation
    # m = np.zeros(vertex_count)  # One mass value for each vertex
    # geom_stiffness = spring_forces / rest_lengths.flatten()  # Initial stiffness
    # for node in range(vertex_count):
    #     adjoining_edges = np.sum((edges == node), axis=1)  # Boolean array for edges connected to the node
    #     stiffness_sum = np.sum((k / rest_lengths.flatten()) * adjoining_edges +
    #                         geom_stiffness * adjoining_edges)
    #     m[node] = stiffness_sum
    #     #m[m == 0] = 1e-6  # Avoid division by zero for isolated nodes

    # Sum forces on vertices
    net_force = np.zeros_like(xyz)
    np.add.at(net_force, edges[:, 0], +edge_force_vectors)
    np.add.at(net_force, edges[:, 1], -edge_force_vectors)

    # Apply external loads and boundary conditions
    net_force[free_indices] += p[free_indices]
    net_force[fixed_indices] = 0  # ensure fixed vertices do not move

    # Update velocities
    a = net_force / m#m[:, None]  # Use adaptive mass
    v[free_indices] += a[free_indices] * dt  # update velocity for free nodes

    # Update positions
    xyz[free_indices] += v[free_indices] * dt

    # Compute kinetic energy
    kinetic_energy = 0.5 * np.sum(m * np.linalg.norm(v, axis=1) ** 2)
    KEStore.append(kinetic_energy)

    # After updating positions, velocities, and kinetic energy, store the history
    history["xyz"].append(xyz.copy())  # Store the current positions
    history["v"].append(v.copy())      # Store the current velocities (optional)
    history["KE"].append(kinetic_energy)  # Store the current kinetic energy
    history["iters"] = iteration     # Store the total number of iterations in the history dictionary

    # Backtracking correction for kinetic energy spike
    if iteration >= 2:  # Ensure sufficient history exists
        if len(KEStore) > 3:
            KEStore.pop(0)
        if kinetic_energy > kinetic_energy_prev and not ResetFlag:
            E = KEStore[-1] - KEStore[-2]  # KE difference between last two iterations
            D = KEStore[-2] - KEStore[-3]  # KE difference before that
            q = max(0, min(E / (E - D), 1))  # Clamp q between 0 and 1

            # Backtrack positions using velocity history
            if len(vStore) >= 1:  # Ensure velocity history is available
                #xyz[free_indices] -= dt * (1 + q) * vStore[-1][free_indices]
                #xyz[free_indices] += (dt**2 / 2) * q * (net_force[free_indices] / m[free_indices])
                R_t = m * (v - vStore[-2]) / dt#m[:, np.newaxis] * (v - vStore[-2]) / dt
                xyz[free_indices] -= (dt * (1 + q) * vStore[-2][free_indices] +
                              (dt ** 2 / 2) * q * (R_t[free_indices] / m))#m[free_indices, np.newaxis]))

            
            # Reset variables
            v[free_indices] = np.zeros_like(v[free_indices])  # Reset velocities
            net_force[free_indices] = np.zeros_like(net_force[free_indices])  # Reset forces
            ResetFlag = True
            print(f"Backtracking at iteration {iteration} due to KE spike")
        else:
            ResetFlag = False  # Allow normal progression if no spike

    # Compute displacement difference for convergence
    position_diff = np.linalg.norm(xyz - xyz_prev, ord='fro')
    if position_diff < tolerance:
        print(f"Converged after {iteration} iterations with displacement difference: {position_diff:.5f}")
        break

    # Store previous positions for next iteration
    xyz_prev = xyz.copy()
    vStore.append(v.copy())
    if len(vStore) > 3:  # Limit memory usage by trimming history
        vStore.pop(0)

    # Update kinetic energy for next iteration
    kinetic_energy_prev = kinetic_energy
    iteration += 1

# Output the results
print(f"Final kinetic energy: {kinetic_energy}")
print(f"Previous kinetic energy: {kinetic_energy_prev}")
print(f"Total iterations: {iteration}")


In [ ]:
import numpy as np

# Input Data
#verts = np.array(v)  # list of initial vertex positions as xyz coordinates
#edges = np.array(e)  # list of edges as pairs of vertex indices
#fixed = list(fixed_indices)  # list of fixed vertex indices
externalLoad = np.zeros((len(verts), 3))  # external load on each vertex (e.g., gravity)

# Parameters
k = 1000  # Hooke's law elasticity constant
dt = 0.09  # time step for the simulation
m = 2  # mass of each vertex
tolerance = 1e-3  # kinetic energy convergence tolerance
max_iterations = 200  # maximum number of iterations

# Derived Data
free_indices = AllFreeIndices#verts[~fixed_indices,:]#list(set(range(len(verts))) - set(fixed))  # indices of free vertices
#print(free_indices)
vertex_count = len(verts)
edge_count = len(edges)
fixed_indices = AllFixedIndices 

# Initialize state variables
xyz = np.copy(verts)  # current vertex positions
v = np.zeros((vertex_count, 3))  # velocity of vertices
#edge_vectors = xyz[edges[:, 1]] - xyz[edges[:, 0]]
#rest_lengths = np.linalg.norm(edge_vectors, axis=1, keepdims=True)  # rest lengths of edges
p = np.copy(externalLoad)  # external forces on vertices

# Initialize tracking variables
vStore = []
KEStore = []
ResetFlag = False

# Main DR Loop
iteration = 0
kinetic_energy_prev = np.inf
xyz_prev = xyz.copy()

while iteration < max_iterations:
    # Compute forces on edges
    edge_vectors_t = xyz[edges[:, 1]] - xyz[edges[:, 0]]
    edge_lengths_t = np.linalg.norm(edge_vectors_t, axis=1, keepdims=True)
    #print(edge_lengths_t)
    spring_forces = k * (edge_lengths_t - rest_lengths)
    edge_force_vectors = spring_forces * (edge_vectors_t / edge_lengths_t)

    # Sum forces on vertices
    net_force = np.zeros_like(xyz)
    np.add.at(net_force, edges[:, 0], -edge_force_vectors)
    np.add.at(net_force, edges[:, 1], edge_force_vectors)

    # Apply external loads and boundary conditions
    net_force[free_indices] += p[free_indices]
    net_force[fixed_indices] = 0  # ensure fixed vertices do not move

    # Update velocities
    a = net_force / m  # acceleration
    v[free_indices] += a[free_indices] * dt  # update velocity for free nodes

    # Update positions
    xyz[free_indices] += v[free_indices] * dt

    # Compute kinetic energy
    kinetic_energy = 0.5 * np.sum(m * np.linalg.norm(v, axis=1) ** 2)

    # Check convergence
    KE_Diff = abs(kinetic_energy_prev - kinetic_energy)
    if KE_Diff < tolerance:
        print(f"Converged after {iteration} iterations.")
        break

    kinetic_energy_prev = kinetic_energy
    iteration += 1

# Output the results
print(f"Final kinetic energy: {kinetic_energy}")
print(f"Total iterations: {iteration}")
#print(f"Final vertex positions:\n{xyz}")




In [ ]:
#print(f"Final kinetic energy: {kinetic_energy}")
#print(f"Prev kinetic energy: {kinetic_energy_prev}")
#print( np.sum(m * np.linalg.norm(v, axis=1) ** 2))
#print(np.linalg.norm(v, axis=1))
#print(v)
print(KE_Diff,edge_lengths_t)
#print(net_force)
#print(net_force[fixed_indices])

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def animate_plot(history, edges, fixed):
    """
    Animate the plot of the geometry in real-time without saving, on a single plot.
    
    Parameters:
        history (dict): A dictionary containing the history of xyz positions.
                        Keys: 'xyz' (list of positions), 'iters' (number of iterations).
        edges (ndarray): Array of edges (pairs of vertex indices).
        fixed (list): Indices of fixed vertices.
    """
    # Create a 3D plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Extract history data
    xyz_history = history['xyz']
    num_iterations = history['iters']

    def plot_geometry(ax, vertices, edges, fixed):
        """
        Helper function to plot the geometry at a given time step.
        """
        # Plot edges
        for edge in edges:
            start, end = edge
            line_x = [vertices[start, 0], vertices[end, 0]]
            line_y = [vertices[start, 1], vertices[end, 1]]
            line_z = [vertices[start, 2], vertices[end, 2]]
            ax.plot(line_x, line_y, line_z, color='b', linewidth=1)

        # Highlight fixed nodes
        fixed_coords = vertices[fixed]
        ax.scatter(fixed_coords[:, 0], fixed_coords[:, 1], fixed_coords[:, 2],
                   color='r', s=50, label='Fixed Nodes')

        # Plot all nodes
        ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], color='k', s=20)

    # Animation loop
    for t in range(num_iterations):
        ax.cla()  # Clear the axes for the next frame

        # Plot the geometry at the current time step
        plot_geometry(ax, xyz_history[t], edges, fixed)

        # Update plot title and view angle
        ax.set_title(f"Iteration: {t+1}")
        ax.view_init(elev=15, azim=130)
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")

        # Redraw the plot
        plt.draw()
        plt.pause(0.001)  # Adjust pause for desired animation speed

    plt.show()

# Example usage (assuming history, edges, and fixed are already defined):
animate_plot(history, edges, AllFixedIndices)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting tools


def plot_first_middle_last(history, edges, fixed):
    """
    Plots the first, middle, and last steps of the history on a single plot.
    
    Parameters:
    history (dict): Dictionary containing the time steps data, e.g., {"xyz": [], "v": [], "KE": []}
    edges (ndarray): Array containing edge connectivity (pairs of vertex indices)
    fixed (list): List of fixed vertex indices
    
    Returns:
    None: The function directly plots the results.
    """
    
    # Extract the first, middle, and last time step
    first_step = 0
    last_step = len(history["xyz"]) - 1
    middle_step = len(history["xyz"]) // 2
    xyz_history = history['xyz']
    
    # Create subplots
    #fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig, axes = plt.subplots(1, 3, figsize=(15, 5), subplot_kw={'projection': '3d'})

    
    # List of time steps to plot
    steps = [first_step, middle_step, last_step]
    titles = ['First Step', 'Middle Step', 'Last Step']
    
    # Loop through the selected steps and plot each
    for i, step in enumerate(steps):
        # Extract positions for this step
        xyz = history["xyz"][step]
        
        # Plot the geometry at this step (you can replace 'plot_geometry2' with your specific plotting function)
        ax = axes[i]
        plot_geometry(ax, xyz_history[step], edges, fixed)  # Pass the specific axis for each subplot
        
        # ax.set_title(titles[i])
        # ax.set_xlabel('X')
        # ax.set_ylabel('Y')
        # ax.set_zlabel('Z')
        # ax.grid(True)
        # ax.view_init(azim=130, elev=15)  # Adjust the view angle
    
    # Display the plot
    plt.tight_layout()
    plt.show()

def plot_geometry(ax, vertices, edges, fixed):
        """
        Helper function to plot the geometry at a given time step.
        """
        # Plot edges
        for edge in edges:
            start, end = edge
            line_x = [vertices[start, 0], vertices[end, 0]]
            line_y = [vertices[start, 1], vertices[end, 1]]
            line_z = [vertices[start, 2], vertices[end, 2]]
            ax.plot(line_x, line_y, line_z, color='b', linewidth=1)

        # Highlight fixed nodes
        fixed_coords = vertices[fixed]
        ax.scatter(fixed_coords[:, 0], fixed_coords[:, 1], fixed_coords[:, 2],
                   color='r', label='Fixed Nodes')

        # Plot all nodes
        ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], color='k')
        ax.view_init(azim=45, elev=30)

# Example usage
plot_first_middle_last(history, edges, AllFixedIndices)
